<a href="https://colab.research.google.com/github/wandb/davis-contest/blob/main/colabs/starter_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions and Starter Code for Submitting Results in the DAVIS Contest

In [ ]:
%%capture

!pip install wandb
!pip install --ignore-installed git+https://github.com/wandb/davis-contest.git#egg=contest
!pip install ptflops pytorch_lightning

In [ ]:
import os 

import wandb

import contest
from contest.utils import clips, paths

## 0️⃣ Create a Weights & Biases account if you don't have one.

## 1️⃣ Download the training data from Weights & Biases

In [ ]:
entity = "charlesfrye"
project = "davis"
mode = "train"
tag = "latest"

training_data_artifact_name = os.path.join(entity, project, f"davis2016-{mode}") + ":" + tag
training_data_artifact_name

In [ ]:
with wandb.init(project=project, job_type="download") as run:
  training_data_artifact = run.use_artifact(training_data_artifact_name)
  training_data_dir = training_data_artifact.download()
  print("\ntraining data downloaded to " + training_data_dir)

### Viewing the Dataset in Weights & Biases

Link to dsviz version, include screenshots.

## 2️⃣ Define and train a model on the data

### Splitting up the data

In [ ]:
print(clips.split_on_clips.__doc__)

First, set up the validation split, at a clipwise level.

In [ ]:
def log_holdout_split(data_artifact, train_split_df, holdout_split_df):
  log_datasplit_artifact(data_artifact, train_split_df, "train")
  log_datasplit_artifact(data_artifact, holdout_split_df, "holdout")


def log_datasplit_artifact(data_artifact, split_df, splitname, folder="wandb"):
  dataset_artifact = wandb.Artifact(name=f"davis2016-split-{splitname}", type="split-data")
  path = os.path.join(folder, splitname + ".json")
  split_df.to_json(path)
  dataset_artifact.add_file(path, "paths.json")

  wandb.run.log_artifact(dataset_artifact)

In [ ]:
config = {"training_fraction": 0.8}

with wandb.init(project=project,
                job_type="split-data", config=config) as run:
  training_data_artifact = run.use_artifact(training_data_artifact_name)
  paths_df = paths.artifact_paths(training_data_artifact)

  training_paths_df, holdout_paths_df = clips.split_on_clips(paths_df)
  log_holdout_split(training_data_artifact,
                    training_paths_df,
                    holdout_paths_df)

### Model Code

In [ ]:
import tensorflow.keras as keras

In [ ]:
def make_model(config):
  model = keras.Sequential(name=config.name)
  
  model.add(keras.layers.InputLayer(input_shape=(None, None, 3)))
  model.add(keras.layers.experimental.preprocessing.Rescaling(1 / 255.))
  model.add(keras.layers.Conv2D(1, kernel_size=1))
  model.add(keras.layers.Activation("sigmoid"))
  
  model.compile(optimizer="sgd", loss="bce")

  return model

For a more realistic model, see _this notebook_.

### Training Code

#### Training the model

In [ ]:
model_artifact_name = "dummy-baseline-keras"

In [ ]:
config = {"batch_size": 32,
          "max_epochs": 1,
          "name": model_artifact_name}

with wandb.init(project=project, config=config, job_type="train") as run:

  config = wandb.config
  training_data_artifact = run.use_artifact(training_data_artifact_name)
  training_data_artifact.download()

  trainsplit_artifact = run.use_artifact("davis2016-split-train:latest")
  trainsplit_paths = paths.get_paths(trainsplit_artifact)

  holdoutsplit_artifact = run.use_artifact("davis2016-split-holdout:latest")
  holdoutsplit_paths = paths.get_paths(holdoutsplit_artifact)

  training_data = contest.keras.data.VidSegDatasetSequence(
    trainsplit_paths["raw"], trainsplit_paths["annotation"], batch_size=config.batch_size)
  holdout_data = contest.keras.data.VidSegDatasetSequence(
    holdoutsplit_paths["raw"], holdoutsplit_paths["annotation"], batch_size=config.batch_size)

  model = make_model(config=config)

  model.fit(training_data, epochs=config.max_epochs,
            validation_data=holdout_data,
            callbacks=[wandb.keras.WandbCallback()]
  )

  wandb.config["nparams"] = contest.keras.profile.count_params(model)
  wandb.config["nflops"] = contest.keras.profile.count_flops(
    model, training_data[0][0])

  model_artifact_id = contest.keras.utils.save_model_to_artifact(
    os.path.join(wandb.run.dir, "model-best.h5"), model_artifact_name)

## 3️⃣ Run your model on the evaluation data

Once you've run your model on the evaluation data,
there's two steps to submission:

1. Log an "evaluation run" to W&B, using _this notebook_.
2. Submit the results to _the benchmark_.

Describe format of the results.

In [ ]:
evaluation_artifact_name = os.path.join(entity, project, "davis2016-val" +":" + tag)

In [ ]:
model_tag = "latest"

In [ ]:
output_dir = os.path.join("outputs")
!rm -rf output_dir
!mkdir -p {output_dir}

In [ ]:
result_artifact_name = model_artifact_name + "-result"

In [ ]:
with wandb.init(project=project, job_type="run-val") as run:
  evaluation_data_artifact = run.use_artifact(evaluation_artifact_name)
  evaluation_data_paths = paths.artifact_paths(evaluation_data_artifact)

  evaluation_dataset = contest.keras.data.VidSegDatasetSequence(
    evaluation_data_paths["raw"])
  num_images = len(evaluation_dataset.image_paths)

  model = contest.keras.utils.load_model_from_artifact(
    model_artifact_name + ":" + model_tag)

  print("\n")
  nparams = model.count_params()
  nflops = contest.keras.profile.count_flops(model, evaluation_dataset[0])

  wandb.log({"nparams": nparams, "nflops": nflops})

  output_paths = contest.keras.evaluate.run(
    model, evaluation_dataset, num_images, output_dir)

  result_artifact = contest.evaluate.make_result_artifact(
    output_paths, result_artifact_name)
  run.log_artifact(result_artifact)

## 4️⃣ Submit your results to the leaderboard on Weights & Biases

Once you've run an evaluation job like the one above and produced a results Artifact,
you're almost ready to submit to the contest.

Head over to _this notebook_ for the last two steps.